In [7]:
import pandas as pd
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time

In [8]:
URL_PREFIXO = 'https://www.mylaptime.com/laptime/clientes/9T00V814PC29X0160812G781W/results/'
URL_PRINCIPAL = 'r3.html?evt=11635&epg=9185'

In [9]:
def get_urls(url):
   
        tomada = None
        corrida = None

        options = webdriver.ChromeOptions()
        options.add_argument('--headless')
        driver = webdriver.Chrome(options=options)
        
        driver.get(url)
        time.sleep(1)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        
        links = soup.find_all('a', href=True)
        
        for link in links:
            if "TOMADA DE TEMPO" in link.get_text():
                 tomada = link['href']
                
        for link in links:
            if "CORRIDA" in link.get_text():
                corrida = link['href']

        return tomada, corrida

In [10]:
def get_tabelas(url_tomada, url_corrida):

    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(options=options)

    driver.get(URL_PREFIXO + url_tomada)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_1 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_tomada.replace('r4.html', 'vlts.asp'))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_2 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_corrida)
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_3 = extrair_dados_tabela(soup.find('table', class_='evenodd'))
    
    driver.get(URL_PREFIXO + url_corrida.replace('r4.html', 'vlts.asp'))
    time.sleep(1)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    tabela_4 = extrair_dados_tabela(soup.find('table', class_='evenodd'))

    return {'tomada_tempo': tabela_1, 'tomada_tempo_mapa_de_voltas': tabela_2, 'corrida': tabela_3, 'corrida_mapa_de_voltas': tabela_4}

def extrair_dados_tabela(tabela):
    dados = []

    for linha in tabela.find_all('tr'):
        dados_linha = []
        for celula in linha.find_all(['th', 'td']):
            dados_linha.append(celula.get_text().strip())
        dados.append(dados_linha)
    
    return  pd.DataFrame(dados[1:], columns=dados[0])

In [11]:
def converter_tempo(tempo):
    if tempo == None:
        return None
    elif tempo.count(':') == 1:
        partes = tempo.split(':')
        try:
            minutos = int(partes[0])
            segundos = float(partes[1])
        except ValueError:
            return None
        return minutos * 60 + segundos
    else:
        return tempo

In [12]:
tabelas['corrida_mapa_de_voltas']

NameError: name 'tabelas' is not defined

In [ ]:
url_tomada, url_corrida = get_urls(URL_PREFIXO + URL_PRINCIPAL)

In [ ]:
tabelas = get_tabelas(url_tomada, url_corrida)

In [ ]:
tabelas['corrida']

,POS,#,CAT.,NOME,M.V,T.M.V,VLTS,T.T,D.L.,D.A.,V.M.(km/h)
0,1,34,INDOOR,PEDRO AGUIAR,10,01:19.325,15,00:20:28.280,---,---,"50,57"
1,2,45,INDOOR,TOMAS TAVEIRA RABELO,14,01:19.266,15,00:20:37.839,00:09.559,00:09.559,"50,20"
2,3,14,INDOOR,MURILO XAVIER,4,01:19.259,15,00:20:48.872,00:20.592,00:11.033,"49,76"
3,4,1,INDOOR,BRUNO FARIA ALMADA,15,01:18.504,15,00:20:53.470,00:25.190,00:04.598,"49,56"
4,5,18,INDOOR,RAFAEL ANGELO VIEIRA PESSOA LIMA,15,01:20.859,15,00:21:17.140,00:48.860,00:23.670,"48,63"
5,6,28,INDOOR,ENRIQUE FERNANDO LIMBERGER,15,01:20.986,15,00:21:21.972,00:53.692,00:04.832,"48,48"
6,7,32,INDOOR,ELTON LAGROTERIA DOS SANTOS,12,01:19.892,15,00:21:24.245,00:55.965,00:02.273,"48,36"
7,8,5,INDOOR,AYRTON MENDONCA,13,01:21.791,15,00:21:25.732,00:57.452,00:01.487,"48,33"
8,9,8,INDOOR,GILBERTO JOSÉ ROSSA JUNIOR,13,01:23.235,14,00:20:30.979,-1 VOLTA(S),-1 VOLTA(S),"47,12"
9,10,42,INDOOR,ALEX NEVES,12,01:22.019,14,00:20:37.162,-1 VOLTA(S),00:06.183,"46,86"


In [ ]:
def converter_mapa_voltas(df):
    colunas_tempo = df.columns.difference(['VLT/COMP'])

    for coluna in colunas_tempo:
        df[coluna] = df[coluna].apply(converter_tempo)

def converter_corrida(df):
    df['V.M.(km/h)'] = df['V.M.(km/h)'].apply(lambda x: None if x is None else float(x.replace('.',',')))

In [ ]:
converter_corrida(tabelas['tomada_tempo'])
converter_mapa_voltas(tabelas['tomada_tempo_mapa_de_voltas'])
converter_corrida(tabelas['corrida'])
converter_mapa_voltas(tabelas['corrida_mapa_de_voltas'])

ValueError: could not convert string to float: '---'

In [ ]:
tabelas['corrida']

,POS,#,CAT.,NOME,M.V,T.M.V,VLTS,T.T,D.L.,D.A.,V.M.(km/h)
0,1,34,INDOOR,PEDRO AGUIAR,10,01:19.325,15,00:20:28.280,---,---,"50,57"
1,2,45,INDOOR,TOMAS TAVEIRA RABELO,14,01:19.266,15,00:20:37.839,00:09.559,00:09.559,"50,20"
2,3,14,INDOOR,MURILO XAVIER,4,01:19.259,15,00:20:48.872,00:20.592,00:11.033,"49,76"
3,4,1,INDOOR,BRUNO FARIA ALMADA,15,01:18.504,15,00:20:53.470,00:25.190,00:04.598,"49,56"
4,5,18,INDOOR,RAFAEL ANGELO VIEIRA PESSOA LIMA,15,01:20.859,15,00:21:17.140,00:48.860,00:23.670,"48,63"
5,6,28,INDOOR,ENRIQUE FERNANDO LIMBERGER,15,01:20.986,15,00:21:21.972,00:53.692,00:04.832,"48,48"
6,7,32,INDOOR,ELTON LAGROTERIA DOS SANTOS,12,01:19.892,15,00:21:24.245,00:55.965,00:02.273,"48,36"
7,8,5,INDOOR,AYRTON MENDONCA,13,01:21.791,15,00:21:25.732,00:57.452,00:01.487,"48,33"
8,9,8,INDOOR,GILBERTO JOSÉ ROSSA JUNIOR,13,01:23.235,14,00:20:30.979,-1 VOLTA(S),-1 VOLTA(S),"47,12"
9,10,42,INDOOR,ALEX NEVES,12,01:22.019,14,00:20:37.162,-1 VOLTA(S),00:06.183,"46,86"


In [ ]:
tabelas['tomada_tempo']

,POS,#,CAT.,NOME,M.V,T.M.V,VLTS,T.T,D.L.,D.A.,V.M.(km/h)
0,1,34,INDOOR,PEDRO AGUIAR,3,01:21.826,4,00:05:55.217,---,---,"50,60"
1,2,1,INDOOR,BRUNO FARIA ALMADA,3,01:21.897,4,00:07:25.911,00:00.071,00:00.071,"50,55"
2,3,14,INDOOR,MURILO XAVIER,4,01:24.358,4,00:06:46.018,00:02.532,00:02.461,"49,08"
3,4,45,INDOOR,TOMAS TAVEIRA RABELO,2,01:26.475,4,00:06:34.195,00:04.649,00:02.117,"47,88"
4,5,18,INDOOR,RAFAEL ANGELO VIEIRA PESSOA LIMA,4,01:26.727,4,00:06:15.221,00:04.901,00:00.252,"47,74"
5,6,28,INDOOR,ENRIQUE FERNANDO LIMBERGER,3,01:27.595,4,00:06:41.167,00:05.769,00:00.868,"47,26"
6,7,42,INDOOR,ALEX NEVES,2,01:27.956,3,00:06:31.982,00:06.130,00:00.361,"47,07"
7,8,27,INDOOR,BRENER SILVA,3,01:28.726,4,00:06:23.104,00:06.900,00:00.770,"46,66"
8,9,5,INDOOR,AYRTON MENDONCA,4,01:30.098,4,00:06:24.967,00:08.272,00:01.372,"45,95"
9,10,24,INDOOR,DIOGO DA SILVA MACIEL,1,01:32.539,3,00:06:09.332,00:10.713,00:02.441,"44,74"
